# Machine Learning API Development Notebook

Ce notebook contient le code de brouillon et les tests pour le développement de l'API de machine learning.


## 1. Importation des Bibliothèques

Nous importons les bibliothèques nécessaires pour le projet.


In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import joblib
import requests


## 2. Définition des Fonctions (function.py)

Nous définissons les fonctions utilisées pour le prétraitement des données, l'entraînement du modèle, le chargement du modèle et la prédiction.


In [7]:
def preprocess_data(features, labels=None):
    """
    Function to preprocess input data for training or prediction.
    :param features: List of feature lists.
    :param labels: List of labels (optional, only for training).
    :return: DataFrame for features, Series for labels (if provided).
    """
    X = pd.DataFrame(features)
    y = pd.Series(labels) if labels is not None else None
    return X, y

def train_model(features, labels, model_path="model/model.joblib"):
    """
    Function to train a logistic regression model and save it to a file.
    :param features: DataFrame of features.
    :param labels: Series of labels.
    :param model_path: Path to save the trained model.
    :return: Trained model.
    """
    model = LogisticRegression(max_iter=200)
    model.fit(features, labels)
    joblib.dump(model, model_path)
    return model

def load_model(model_path="model/model.joblib"):
    """
    Function to load a trained model from a file.
    :param model_path: Path to the saved model.
    :return: Loaded model.
    """
    model = joblib.load(model_path)
    return model

def predict(model, features):
    """
    Function to make predictions using a trained model.
    :param model: Trained model.
    :param features: DataFrame of features.
    :return: List of predictions.
    """
    predictions = model.predict(features)
    return predictions.tolist()


## 3. Tests des Fonctions

Nous créons un jeu de données fictif, prétraitons les données, entraînons le modèle, chargeons le modèle et effectuons des prédictions pour vérifier le bon fonctionnement des fonctions.


In [ ]:
# Création d'un jeu de données fictif
features = [
    [2.3, 4.5, 1.2, 0, 1, 0.1, 0.2],
    [3.4, 1.2, 0.5, 1, 0, 0.4, 0.7],
    [1.2, 3.4, 2.2, 0, 1, 0.3, 1.1],
    [0.5, 0.8, 1.5, 1, 0, 0.9, 1.4],
    [1.5, 1.8, 0.9, 0, 1, 0.2, 0.7]
]
labels = [1, 0, 1, 0, 1]

# Prétraitement des données
X, y = preprocess_data(features, labels)
X, y


### Entraînement du modèle


In [ ]:
# Entraînement du modèle
model = train_model(X, y, model_path="model/model.joblib")
model


### Chargement du modèle et prédiction


In [ ]:
# Chargement du modèle et prédiction
loaded_model = load_model(model_path="model/model.joblib")
predictions = predict(loaded_model, X)
predictions


## 4. Utilisation de l'API (app.py)

Nous montrons comment l'API est utilisée pour l'entraînement du modèle et les prédictions.


In [ ]:
import streamlit as st
import requests
import pandas as pd

st.title("Machine Learning API Interface")
st.markdown("""
    ### Documentation
    - [Swagger UI](http://localhost:8000/docs)
    - [ReDoc](http://localhost:8000/redoc)
""")

# Entraînement du modèle
st.header("Train Model")

uploaded_file = st.file_uploader("Upload CSV", type=["csv"])

if uploaded_file is not None:
    data = pd.read_csv(uploaded_file)
    st.write("Data preview:", data.head())

    if st.button("Train Model"):
        features = data.iloc[:, :-1].values.tolist()
        labels = data.iloc[:, -1].tolist()

        response = requests.post("http://localhost:8000/training", json={"features": features, "labels": labels})
        
        if response.status_code == 200:
            st.success("Model trained successfully")
        else:
            st.error("Error training model: " + response.text)

# Prédiction
st.header("Make Prediction")
st.write("Example input format: `6, 148, 72, 35, 0, 33.6, 0.627, 50`")

prediction_data = st.text_area("Enter data for prediction (comma-separated values for each feature)")

if 'predictions' not in st.session_state:
    st.session_state['predictions'] = None

if st.button("Predict"):
    try:
        features = [list(map(float, prediction_data.split(',')))]
        st.write("Sending features to API:", features)
        response = requests.post("http://localhost:8000/predict", json={"features": features})
        
        if response.status_code == 200:
            st.session_state['predictions'] = response.json()["predictions"]
        else:
            st.error("Error making prediction: " + response.text)
    except ValueError as e:
        st.error(f"Please enter valid data: {e}")

# Affichage des prédictions si elles existent
if st.session_state['predictions'] is not None:
    st.write("Prediction:", st.session_state['predictions'])

st.header("Model Info")
response = requests.get("http://localhost:8000/model")

if response.status_code == 200:
    model_info = response.json()
    st.write("Model:", model_info["model"])
    st.write("Library:", model_info["library"])
else:
    st.error("Error fetching model info: " + response.text)


## 5. Tests API

Nous testons les endpoints de l'API en envoyant des requêtes POST pour entraîner le modèle et faire des prédictions.


In [ ]:
# Test de l'API d'entraînement
training_data = {
    "features": features,
    "labels": labels
}
response = requests.post("http://localhost:8000/training", json=training_data)
print("Training response:", response.json())


In [ ]:
# Test de l'API de prédiction
prediction_data = {
    "features": features
}
response = requests.post("http://localhost:8000/predict", json=prediction_data)
print("Prediction response:", response.json())
